In [1]:
import os
import pymaster
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import skyproj
import healsparse as hsp
from scipy.interpolate import interp1d

import matplotlib
matplotlib.style.use('des_dr1')

In [2]:
planck_mask_file = '/n/netscratch/dvorkin_lab/Lab/kboone/cmb_conv/masks/Planck.npy'
mask = np.load(planck_mask_file)

In [3]:
conv_dir = '/n/netscratch/dvorkin_lab/Lab/gvalogiannis/maps_cmbl_georgios/maps_cmbl_georgios/'
fid_file = conv_dir + 'fiducial_0.npy'
noiseless_map = np.load(fid_file, allow_pickle=True).item()['full_sky_noiseless']

In [4]:
fwhm_arcmin = 10  # FWHM in arcminutes
fwhm_rad = np.radians(fwhm_arcmin / 60.0)  # Convert to radians
nside = 1024
nside_ls = nside*3

N = 1e7
x = [10,75.32337589203746	,160.85999889800618	,340.4967383339498	,871.2051805682871	,1724.809488637684	]
y = [1,1.0182,1.1348,1.4041,1.5198,2.5557	]
f = interp1d(x, y, kind='linear', fill_value="extrapolate")
nn = hp.synfast(2*f(np.arange(nside_ls))/N,nside =1024)

In [5]:
cmb_map = noiseless_map + nn
cmb_map = hp.sphtfunc.smoothing(cmb_map, fwhm=fwhm_rad)

In [6]:
cmb_map[np.where(mask < 0.5)] = hp.UNSEEN

In [7]:
nest_map = hp.reorder(cmb_map, inp='RING', out='NEST')
hsp_map = hsp.HealSparseMap(nside_coverage=32, healpix_map=nest_map)

In [8]:
fig, ax = plt.subplots(figsize=(8, 6))
sp = skyproj.McBrydeSkyproj(ax=ax) 

# ——— main map ——————————————————————————————————————————————
# draw_hspmap returns the QuadMesh as its first element
quadmesh, *_ = sp.draw_hspmap(
    hsp_map,
    cmap='viridis',
    edgecolors='none',      # no per-cell borders at all
    linewidth=0,
    antialiased=False)

quadmesh.set_rasterized(True)   # ← ***this one line removes the seams***

sp.ax.set_xlabel(sp.ax.get_xlabel(), fontsize=24)
sp.ax.set_ylabel(sp.ax.get_ylabel(), fontsize=24)
plt.colorbar(location = 'right', fraction = 0.02)
fig.suptitle(r'$\kappa_\text{CMB}$ with Planck Mask and Noise', y = 0.825, size=24)

# save: dpi controls the raster resolution inside the PDF
plt.savefig('../Paper_Plots/Example_Map.pdf', format='pdf', bbox_inches='tight', dpi=400)
plt.close()

In [9]:
# fig = plt.figure(1, figsize=(10, 8))
# ax = fig.add_subplot(111)
# sp = skyproj.McBrydeSkyproj(ax=ax)
# sp.draw_hspmap(hsp_map)
# sp.ax.set_xlabel(sp.ax.get_xlabel(), fontsize=20)
# sp.ax.set_ylabel(sp.ax.get_ylabel(), fontsize=20)
# plt.colorbar(location = 'right', fraction = 0.02)
# fig.suptitle(r'$\kappa_\text{CMB}$ with Planck Mask and Noise', y = 0.8, size=24)
# plt.show()